# Task Delegation with a Worker for Prompt Refinement

This notebook demonstrates a pattern where an "Initial Agent" delegates a sub-task to a "Task Worker." The Task Worker uses its own AI engine to perform meaningful prompting (e.g., refining a query, generating context, or suggesting improvements). The worker's output is then returned to the Initial Agent, which uses it to build a better, more detailed prompt for its primary task execution.

## 1. Setup

Import necessary libraries and AILF components. This includes `AIEngine` for interacting with LLMs, Pydantic for data validation, and other standard Python libraries. We'll also load environment variables for API keys.

In [ ]:
# Standard library imports
import os
import asyncio
from typing import Dict, Any, List, Optional
from pprint import pprint

# Pydantic for data modeling
from pydantic import BaseModel, Field

# AILF framework components
from ailf.ai.engine import AIEngine # Assuming AIEngine is in this path
from ailf.schemas.ai import AIConfig # Assuming AIConfig is in this path

# Load environment variables (for API keys)
from dotenv import load_dotenv
load_dotenv()

# Check for API keys (OpenAI and Anthropic)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

api_keys_available = {
    "openai": bool(OPENAI_API_KEY),
    "anthropic": bool(ANTHROPIC_API_KEY)
}

print("API Keys Available:")
for api, available in api_keys_available.items():
    print(f"- {api.upper()}: {'✓' if available else '✗'}")

if not any(api_keys_available.values()):
    print("\nWarning: No API keys found. LLM calls will fail.")
    print("Please set up your .env file with OPENAI_API_KEY or ANTHROPIC_API_KEY.")

### Helper Function to Create AI Engine
This function will help us easily create `AIEngine` instances for our agent and worker.

In [ ]:
async def create_ai_engine(default_provider: str = "openai", model: Optional[str] = None) -> Optional[AIEngine]:
    """Creates and initializes an AI Engine with the specified provider."""
    config_dict = {
        "default_provider": None,
        "log_prompts": True,
        "log_responses": True,
        "providers": {}
    }

    if api_keys_available["openai"]:
        config_dict["providers"]["openai"] = {
            "api_key": OPENAI_API_KEY,
            "default_model": model if model and default_provider == "openai" else "gpt-4o-mini",
            "default_temperature": 0.7,
            "timeout": 60,
            "enabled": True
        }
        if not config_dict["default_provider"]:
             config_dict["default_provider"] = "openai"

    if api_keys_available["anthropic"]:
        config_dict["providers"]["anthropic"] = {
            "api_key": ANTHROPIC_API_KEY,
            "default_model": model if model and default_provider == "anthropic" else "claude-3-haiku-20240307",
            "default_temperature": 0.7,
            "timeout": 60,
            "enabled": True
        }
        if not config_dict["default_provider"]:
            config_dict["default_provider"] = "anthropic"
    
    # Override default provider if specified and available
    if default_provider in config_dict["providers"] and config_dict["providers"][default_provider]["enabled"]:
        config_dict["default_provider"] = default_provider
    elif not config_dict["default_provider"]:
        print(f"Warning: Specified default provider '{default_provider}' is not available or configured.")
        print(f"No API keys available to create an AI Engine.")
        return None

    ai_config = AIConfig(**config_dict)
    engine = AIEngine(config=ai_config)
    await engine.initialize()
    print(f"AI Engine created with default provider: {engine.config.default_provider} using model {engine.get_default_model()}\n")
    return engine

## 2. Define Pydantic Models for Structured Output

We'll use Pydantic models to define the expected structure of the data returned by our Task Worker and the final output.

In [ ]:
class RefinedQueryComponents(BaseModel):
    """Structure for components refined by the Task Worker."""
    main_goal: str = Field(description="The core objective of the request.")
    key_elements_to_include: List[str] = Field(description="Specific elements or topics that must be part of the response.")
    target_audience: Optional[str] = Field(default=None, description="The intended audience for the final output.")
    desired_tone: Optional[str] = Field(default=None, description="The preferred tone or style of the response (e.g., formal, creative, humorous).")
    constraints_or_exclusions: List[str] = Field(default_factory=list, description="Any constraints or topics to avoid.")

class FinalContentOutput(BaseModel):
    """Structure for the final content generated by the Initial Agent."""
    title: str = Field(description="A suitable title for the generated content.")
    content_body: str = Field(description="The main body of the generated content.")
    suggestions_for_next_steps: Optional[List[str]] = Field(default=None, description="Optional suggestions for how to use or extend this content.")

## 3. Define the Task Worker

The `TaskWorker` will take an initial, potentially vague, user query. Its job is to use its AI engine to break down this query into more structured components, making it easier for the `InitialAgent` to formulate a high-quality prompt.

In [ ]:
class TaskWorker:
    """A worker that refines a user query into structured components."""
    def __init__(self, ai_engine: AIEngine):
        self.ai_engine = ai_engine
        if not self.ai_engine:
            raise ValueError("TaskWorker requires a valid AIEngine instance.")

    async def refine_query(self, user_query: str) -> Optional[RefinedQueryComponents]:
        """Uses AI to refine the user query into structured components."""
        system_prompt = (
            "You are an expert query analyst. Your task is to take a user's request and break it down "
            "into structured components that will help another AI generate a high-quality response. "
            "Identify the main goal, key elements to include, target audience (if inferable), desired tone (if inferable), "
            "and any implicit constraints or items to exclude."
        )
        
        prompt = f"Analyze the following user request and extract its structured components:\n\nUser Request: {user_query}"
        
        print(f"\n🤖 Task Worker: Refining query - '{user_query[:50]}...'\n")
        try:
            response = await self.ai_engine.generate_structured(
                system=system_prompt,
                user=prompt,
                output_schema=RefinedQueryComponents
            )
            if response and response.output:
                print("\n🤖 Task Worker: Query refined successfully.")
                pprint(response.output.model_dump(), indent=2)
                return response.output
            else:
                print("\n🤖 Task Worker: Failed to get structured output for query refinement.")
                return None
        except Exception as e:
            print(f"\n🤖 Task Worker: Error during query refinement - {e}")
            return None

## 4. Define the Initial Agent

The `InitialAgent` receives a high-level goal. It first delegates to the `TaskWorker` to get refined query components. Then, it uses these components to construct a detailed prompt for its own AI engine to generate the final desired output.

In [ ]:
class InitialAgent:
    """An agent that uses a TaskWorker to refine prompts before execution."""
    def __init__(self, ai_engine: AIEngine, task_worker: TaskWorker):
        self.ai_engine = ai_engine
        self.task_worker = task_worker
        if not self.ai_engine or not self.task_worker:
            raise ValueError("InitialAgent requires valid AIEngine and TaskWorker instances.")

    def _construct_final_prompt(self, refined_components: RefinedQueryComponents) -> str:
        """Constructs a detailed final prompt from refined components."""
        prompt_parts = [
            f"Objective: {refined_components.main_goal}",
            "\nKey Elements to Incorporate:",
            *["- " + item for item in refined_components.key_elements_to_include],
        ]
        if refined_components.target_audience:
            prompt_parts.append(f"\nTarget Audience: {refined_components.target_audience}")
        if refined_components.desired_tone:
            prompt_parts.append(f"\nDesired Tone: {refined_components.desired_tone}")
        if refined_components.constraints_or_exclusions:
            prompt_parts.append("\nConstraints/Exclusions:")
            prompt_parts.extend(["- " + item for item in refined_components.constraints_or_exclusions])
        
        prompt_parts.append("\nPlease generate the content based on these detailed instructions.")
        return "\n".join(prompt_parts)

    async def process_goal(self, user_goal: str) -> Optional[FinalContentOutput]:
        """Processes a user goal by delegating refinement and then generating content."""
        print(f"\n🚀 Initial Agent: Received goal - '{user_goal[:50]}...'\n")
        
        # 1. Delegate to Task Worker for query refinement
        refined_components = await self.task_worker.refine_query(user_goal)
        if not refined_components:
            print("\n🚀 Initial Agent: Could not get refined components from Task Worker. Aborting.")
            return None
        
        # 2. Construct the final detailed prompt
        final_prompt = self._construct_final_prompt(refined_components)
        print("\n🚀 Initial Agent: Constructed final prompt:")
        print("-" * 30)
        print(final_prompt)
        print("-" * 30)
        
        # 3. Use own AI engine to generate the final output
        system_prompt_final_generation = (
            "You are a creative content generation AI. Your task is to produce high-quality content "
            "based on the detailed instructions provided. Ensure the output is well-structured and engaging."
        )
        
        print("\n🚀 Initial Agent: Generating final content...")
        try:
            response = await self.ai_engine.generate_structured(
                system=system_prompt_final_generation,
                user=final_prompt,
                output_schema=FinalContentOutput
            )
            if response and response.output:
                print("\n🚀 Initial Agent: Final content generated successfully.")
                pprint(response.output.model_dump(), indent=2)
                return response.output
            else:
                print("\n🚀 Initial Agent: Failed to get structured output for final content generation.")
                return None
        except Exception as e:
            print(f"\n🚀 Initial Agent: Error during final content generation - {e}")
            return None

## 5. Demonstration

Let's instantiate the agent and worker and see them in action. We'll provide a sample high-level goal.

In [ ]:
async def main():
    # Create AI engines for worker and agent
    # You might use different models or configurations for each
    worker_ai_engine = await create_ai_engine(default_provider="openai", model="gpt-4o-mini") # Worker might use a faster/cheaper model for analysis
    agent_ai_engine = await create_ai_engine(default_provider="openai", model="gpt-4o-mini")  # Agent might use a more powerful model for generation

    if not worker_ai_engine or not agent_ai_engine:
        print("Could not create AI engines. Ensure API keys are set and providers are available. Aborting demonstration.")
        return

    # Instantiate worker and agent
    task_worker_instance = TaskWorker(ai_engine=worker_ai_engine)
    initial_agent_instance = InitialAgent(ai_engine=agent_ai_engine, task_worker=task_worker_instance)

    # Define a sample user goal
    user_goal = "I need a short blog post about the benefits of remote work for small businesses, targeting entrepreneurs who are hesitant to adopt it. Make it encouraging but also acknowledge potential challenges briefly."
    
    # Process the goal
    final_output = await initial_agent_instance.process_goal(user_goal)
    
    if final_output:
        print("\n🎉 Demonstration Complete! Final Output:")
        # Output is already pretty-printed by the agent, but we can show specific parts here if needed
        # print(f"Title: {final_output.title}")
        # print(f"Content: {final_output.content_body[:200]}...")
    else:
        print("\n😔 Demonstration Failed to produce final output.")
    
    # Clean up AI Engines (if they have a shutdown method)
    if hasattr(worker_ai_engine, 'shutdown'):
        await worker_ai_engine.shutdown()
    if hasattr(agent_ai_engine, 'shutdown'):
        await agent_ai_engine.shutdown()

# Run the demonstration
# Ensure you have an event loop running if you are in a script.
# In Jupyter, this will typically run fine.
if __name__ == '__main__':
    # In a .py script, you would run asyncio.run(main())
    # In Jupyter, we can often await main() directly if the loop is managed, 
    # or use nest_asyncio if needed, but let's try direct await for simplicity first.
    # For robust execution in Jupyter, especially if other async code is running:
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())

## 6. Conclusion

This notebook demonstrated a task delegation pattern where an `InitialAgent` leverages a `TaskWorker` to refine a user's request into a more structured and detailed set of components. This allows the `InitialAgent` to construct a higher-quality prompt for its own AI engine, leading to better final outputs.

**Benefits of this pattern:**
- **Improved Prompt Quality**: Breaking down the task allows for more focused and detailed prompt engineering.
- **Modularity**: The worker and agent can be developed and scaled independently. Different workers could specialize in different types of refinement.
- **Specialization**: The worker's AI can be optimized (e.g., different model, specific system prompt) for analytical/refinement tasks, while the agent's AI can be optimized for generative tasks.
- **Reduced Cognitive Load on Agent**: The agent doesn't need to handle the initial ambiguity directly; it receives a more structured input.
- **Potential for Cost Optimization**: The refinement task might be achievable with a less powerful (and cheaper) LLM, reserving the more powerful LLM for the final generation.